In [1]:
import warnings
from crimm import fetch_rcsb
from crimm.StructEntities.OrganizedModel import OrganizedModel
from crimm.Modeller.Solvator import Solvator

from crimm.Fetchers import fetch_rcsb
from crimm.Modeller import TopologyGenerator
from crimm.Modeller.CoordManipulator import CoordManipulator
from crimm.Modeller.LoopBuilder import ChainLoopBuilder
from crimm.Adaptors.PropKaAdaptors import PropKaProtonator
from crimm.Utils.StructureUtils import get_coords

import pycharmm
from pycharmm.settings import set_verbosity as pcm_set_verbosity
from pycharmm import write as pcm_write
from pycharmm import NonBondedScript

from crimm.Adaptors.pyCHARMMAdaptors import (
    load_chain, load_topology, load_water, load_ions, load_ligands,
    create_water_hs_from_charmm, fetch_coords_from_charmm, patch_disu_from_model,
    sd_minimize
)

import pycharmm.minimize as minimize
import pycharmm.energy as energy
from pycharmm import coor, crystal, image, cons_harm, cons_fix, generate

In [2]:
# cgenff excutable path is used later in topology generation
CGENFF_PATH = "/export/app/cgenff/silcsbio.2024.1/cgenff/cgenff"
PDBID = '5igv' #'5iev'#'1bg8' #'3q4k' #'4pti' #'2HZI' 

## Fetch from RCSB

The fetch_rcsb has be updated that it takes argument `organize`. When it is `True`, the structure will be organized into chain types, and an `OrganizedModel` will be returned instead of the unorganized structure entity.

In [3]:
model = fetch_rcsb(
    PDBID,
    include_solvent=True, # We want to incude crystallographic water
    use_bio_assembly=True,
    organize=True
)

/home/truman/crimm/crimm/StructEntities/Model.py:110: UserWarning: Chain F is a solvent chain, connect atom assignment skipped!
  warnings.warn(
/home/truman/crimm/crimm/StructEntities/Model.py:84: UserWarning: Insufficient atoms for connect record metalc
  warnings.warn(
/home/truman/crimm/crimm/StructEntities/Model.py:110: UserWarning: Chain E is a solvent chain, connect atom assignment skipped!
  warnings.warn(
/home/truman/crimm/crimm/StructEntities/Model.py:84: UserWarning: Insufficient atoms for connect record metalc
  warnings.warn(
/home/truman/crimm/crimm/StructEntities/Model.py:116: UserWarning: Residue None not found in chain A
  warnings.warn(
/home/truman/crimm/crimm/StructEntities/Model.py:99: UserWarning: Chain F not found in model 1
  warnings.warn(


In [4]:
## the OrganinzedModel is improved with more feature and APIs
## and has become the main object that deals with modeling and interfacing pyCHARMM
## There will be another notebook showcasing more about OrganizedModel
model

NGLWidget()

<OrganizedModel model=5IGV Polypeptide(L)=1 Solvent=1 NucleosidePhosphate=1 Ligand=1 Ion=1 >
	│
	├───<Polypeptide(L) id=A Residues=298>
	├──────Description: Macrolide 2'-phosphotransferase II
	│
	├───<NucleosidePhosphate id=B Molecules=1>
	├──────Residue ID(s): GDP
	├──────Description: GUANOSINE-5'-DIPHOSPHATE
	│
	├───<Ligand id=C Molecules=1>
	├──────Residue ID(s): ZIT
	├──────Description: AZITHROMYCIN
	│
	├───<Ion id=D Residues=2>
	├──────Residue ID(s): CA, MG
	├──────Description: MAGNESIUM ION, CALCIUM ION
	│
	├───<Solvent id=E Residues=456>
	├──────Residue ID(s): HOH
	├──────Description: water


In [5]:
## Place the model center to (0, 0, 0) and place the principle axis along x-axis
coord_man = CoordManipulator()
coord_man.load_entity(model)
coord_man.orient_coords()

In [6]:
model

NGLWidget()

<OrganizedModel model=5IGV Polypeptide(L)=1 Solvent=1 NucleosidePhosphate=1 Ligand=1 Ion=1 >
	│
	├───<Polypeptide(L) id=A Residues=298>
	├──────Description: Macrolide 2'-phosphotransferase II
	│
	├───<NucleosidePhosphate id=B Molecules=1>
	├──────Residue ID(s): GDP
	├──────Description: GUANOSINE-5'-DIPHOSPHATE
	│
	├───<Ligand id=C Molecules=1>
	├──────Residue ID(s): ZIT
	├──────Description: AZITHROMYCIN
	│
	├───<Ion id=D Residues=2>
	├──────Residue ID(s): CA, MG
	├──────Description: MAGNESIUM ION, CALCIUM ION
	│
	├───<Solvent id=E Residues=456>
	├──────Residue ID(s): HOH
	├──────Description: water


In [7]:
model['B']

NGLWidget()

<NucleosidePhosphate id=B Molecules=1>
  Residue ID(s): GDP
  Description: GUANOSINE-5'-DIPHOSPHATE


In [8]:
# build missing loops if exist
for chain in model.protein:
    if not chain.is_continuous():
        # chain can be built in place now by specifying `inplace = True`
        looper = ChainLoopBuilder(chain, inplace = True)
        # looper.build_from_homology(max_num_match=10, identity_score_cutoff=0.95)
        # missing terminals will also be built if `include_terminal = True`
        looper.build_from_alphafold(include_terminal = False)

In [9]:
chain.is_continuous()

True

## Generate Topology

Topology generation is simplified by using organized model. If `cgenff_path` is specified, ligands are also generated

In [10]:
topo = TopologyGenerator(
    cgenff_excutable_path=CGENFF_PATH,
    cgenff_output_path='./cgenff/'
)
topo.generate_model(
    model,
    prot_first_patch='ACE',
    prot_last_patch='CT3',
    coerce=True
)

/home/truman/crimm/crimm/Modeller/TopoLoader.py:738: UserWarning: No cmap found in <Polypeptide(L) id=A Residues=298>.
  warnings.warn(


### Printing out the TOPPAR and their Versions Being Used and Loaded

In [11]:
for rtf_type, topo_loader in topo.res_def_dict.items():
    print(rtf_type, 'toppar version:', topo_loader.rtf_version)

cgenff toppar version: 36.1
protein toppar version: 36.2
water_ions toppar version: 31.1


In [12]:
TopologyGenerator?

Init signature: TopologyGenerator(cgenff_excutable_path=None, cgenff_output_path=None)
Docstring:     
Class for generating topology elements from the topology definition
and parameters.
The topology definition the parameters are loaded from the CHARMM 36 RTF and 
PRM files.
If the cgenff_excutable_path is provided, the topology definition and 
parameters for the heterogen residues are generated by cgenff, and ligand mol2
file and cgenff rtf file will be saved if cgenff_output_path is specified.
Init docstring: Initialize the topology generator.
File:           ~/crimm/crimm/Modeller/TopoLoader.py
Type:           type
Subclasses:     

In [13]:
# Organized model
model

NGLWidget()

<OrganizedModel model=5IGV Polypeptide(L)=1 Solvent=1 NucleosidePhosphate=1 Ligand=1 Ion=1 >
	│
	├───<Polypeptide(L) id=A Residues=298>
	├──────Description: Macrolide 2'-phosphotransferase II
	│
	├───<NucleosidePhosphate id=B Molecules=1>
	├──────Residue ID(s): GDP
	├──────Description: GUANOSINE-5'-DIPHOSPHATE
	│
	├───<Ligand id=C Molecules=1>
	├──────Residue ID(s): ZIT
	├──────Description: AZITHROMYCIN
	│
	├───<Ion id=D Residues=2>
	├──────Residue ID(s): CA, MG
	├──────Description: MAGNESIUM ION, CALCIUM ION
	│
	├───<Solvent id=E Residues=456>
	├──────Residue ID(s): HOH
	├──────Description: water


Modified residue creates breaks in chain after coersion

In [14]:
# Protonation 
protonator = PropKaProtonator(topo, pH = 7.4)
protonator.load_model(model)
# if there is any pathching applied in crimm, CHARMM PATCH command will be automatically run 
# when protein chains are loaded into CHARMM
protonator.apply_patches()

Unexpected number (9) of atoms in residue SER   2 A   in conformation 1A
Unexpected number (9) of atoms in residue VAL 299 A   in conformation 1A


No protonation patches to apply on chain A.


In [15]:
## All the topology definition and parameter generated for the model is 
## organized in model.topology_loader. load_topology() takes care of 
## loading sequence and only loads what is need for the model
load_topology(model.topology_loader)

  
 CHARMM>     read rtf card -
 CHARMM>     name /tmp/tmpbrra2oyk
 VOPEN> Attempting to open::/tmp/tmpbrra2oyk::
 MAINIO> Residue topology file being read from unit  91.
 TITLE> * PROTEIN RTF LOADED FROM CRIMM
 TITLE> 36  2
 VCLOSE: Closing unit   91 with status "KEEP"
  
 CHARMM>     
  
  
 CHARMM>     read param card -
 CHARMM>     name /tmp/tmpf_p5gqzk -
 CHARMM>     flex
 VOPEN> Attempting to open::/tmp/tmpf_p5gqzk::

          PARAMETER FILE BEING READ FROM UNIT 91
 TITLE> * PROTEIN PRM LOADED FROM CRIMM
 TITLE> *>>>> CHARMM36 ALL-HYDROGEN PARAMETER FILE FOR PROTEINS <<<<<<<<<<
 TITLE> *>>>>> INCLUDES PHI, PSI CROSS TERM MAP (CMAP) CORRECTION <<<<<<<<
 TITLE> *>>>>>>>>>>>>>>>>>>>>>>>>>> JAN. 2016 <<<<<<<<<<<<<<<<<<<<<<<<<<<<
 TITLE> * ALL COMMENTS TO THE CHARMM WEB SITE: WWW.CHARMM.ORG
 TITLE> *             PARAMETER SET DISCUSSION FORUM
 TITLE> *
 PARMIO> NONBOND, HBOND lists and IMAGE atoms cleared.
 VCLOSE: Closing unit   91 with status "KEEP"
  
 CHARMM>     
  
  
 CHARMM> 

In [16]:
for chain in model.protein:
    load_chain(chain)
# we need to patch disulfide bonds in CHARMM
# the disu info is stored in model under model.connect_dict
patch_disu_from_model(model)

  
 CHARMM>     read sequence pdb -
 CHARMM>     name /tmp/tmp8fil40tw
 VOPEN> Attempting to open::/tmp/tmp8fil40tw::
 MAINIO> Sequence information being read from unit  91.
 TITLE>  *

          RESIDUE SEQUENCE --   298 RESIDUES
          SER LYS ASP ILE LYS GLN VAL ILE GLU ILE ALA LYS LYS HSD ASN LEU PHE LEU LYS GLU 
          GLU THR ILE GLN PHE ASN GLU SER GLY LEU ASP PHE GLN ALA VAL PHE ALA GLN ASP ASN 
          ASN GLY ILE ASP TRP VAL LEU ARG LEU PRO ARG ARG GLU ASP VAL MET PRO ARG THR LYS 
          VAL GLU LYS GLN ALA LEU ASP LEU VAL ASN LYS TYR ALA ILE SER PHE GLN ALA PRO ASN 
          TRP ILE ILE TYR THR GLU GLU LEU ILE ALA TYR LYS LYS LEU ASP GLY VAL PRO ALA GLY 
          THR ILE ASP HSD ASN ILE GLY ASN TYR ILE TRP GLU ILE ASP ILE ASN ASN VAL PRO GLU 
          LEU PHE HSD LYS SER LEU GLY ARG VAL LEU ALA GLU LEU HSD SER ILE PRO SER ASN LYS 
          ALA ALA ALA LEU ASP LEU VAL VAL HSD THR PRO GLU GLU ALA ARG MET SER MET LYS GLN 
          ARG MET ASP ALA VAL ARG ALA LYS

In [17]:
# model.ligand+model.phos_ligand+model.co_solvent is the concatenated list of entities
load_ligands(model.ligand+model.phos_ligand+model.co_solvent)
# load_ligands(model.ligand)

[crimm] Loading ligand ZIT SEG: LG00
  
 CHARMM>     read sequence pdb -
 CHARMM>     name /tmp/tmp2rxi2ouz
 VOPEN> Attempting to open::/tmp/tmp2rxi2ouz::
 MAINIO> Sequence information being read from unit  91.
 TITLE>  *

          RESIDUE SEQUENCE --     1 RESIDUES
          ZIT     
 VCLOSE: Closing unit   91 with status "KEEP"
  
 CHARMM>     
  
 NO PATCHING WILL BE DONE ON THE FIRST RESIDUE
 NO PATCHING WILL BE DONE ON THE LAST  RESIDUE
 AUTGEN: Autogenerating specified angles and dihedrals.
 GENPSF> Segment   2 has been generated. Its identifier is LG00.
 PSFSUM> PSF modified: NONBOND lists and IMAGE atoms cleared.
 PSFSUM> Summary of the structure file counters :
         Number of segments      =        2   Number of residues   =      299
         Number of atoms         =     4907   Number of groups     =     1418
         Number of bonds         =     4967   Number of angles     =     9008
         Number of dihedrals     =    13188   Number of impropers  =      840
        

['LG00', 'LG01']

## Minimize the Protein Chain First

In [18]:
# Specify nonbonded python object called my_nbonds - this just sets it up
# equivalant CHARMM scripting command: 
# nbonds cutnb 18 ctonnb 13 ctofnb 17 cdie eps 1 atom vatom fswitch vfswitch
non_bonded_script = NonBondedScript(
    cutnb=18.0, ctonnb=13.0, ctofnb=17.0,
    eps=1.0,
    cdie=True,
    atom=True, vatom=True,
    fswitch=True, vfswitch=True
)
# select the C-alpha atoms for harmonic restraints
cons_harm_atoms = pycharmm.SelectAtoms(atom_type='CA')
ener_dict = sd_minimize(300, non_bonded_script, cons_harm_selection=cons_harm_atoms)

  
 CHARMM>     nbonds cutnb 18.0 -
 CHARMM>     ctonnb 13.0 -
 CHARMM>     ctofnb 17.0 -
 CHARMM>     eps 1.0 -
 CHARMM>     cdie -
 CHARMM>     atom -
 CHARMM>     vatom -
 CHARMM>     fswitch -
 CHARMM>     vfswitch

 NONBOND OPTION FLAGS: 
     ELEC     VDW      ATOMs    CDIElec  FSWItch  VATOm    VFSWIt  
     BYGRoup  NOEXtnd  NOEWald 
 CUTNB  = 18.000 CTEXNB =999.000 CTONNB = 13.000 CTOFNB = 17.000
 CGONNB =  0.000 CGOFNB = 10.000
 WMIN   =  1.500 WRNMXD =  0.500 E14FAC =  1.000 EPS    =  1.000
 NBXMOD =      5
 There are        0 atom  pairs and        0 atom  exclusions.
 There are        0 group pairs and        0 group exclusions.
 <MAKINB> with mode   5 found  14096 exclusions and  13045 interactions(1-4)
 <MAKGRP> found   4151 group exclusions.
 Generating nonbond list with Exclusion mode = 5
 == PRIMARY == SPACE FOR  3023098 ATOM PAIRS AND        0 GROUP PAIRS
 NBONDA>>  Maximum group spatial extent (12A) exceeded.
   Size is       14.75 Angstroms and starts with atom:   

/home/truman/crimm/crimm/Adaptors/pyCHARMMAdaptors.py:449: UserWarning: Absolute harmonic restraints setup success: True
  warnings.warn(f"Absolute harmonic restraints setup success: {has_cons_harm}")




 STEEPD> An energy minimization has been requested.

 NSTEP  =          300   NPRINT =           10
 STEP   =    0.0200000   TOLFUN =    0.0010000
 TOLGRD =    0.0010000   TOLSTP =    0.0000000

MINI MIN: Cycle      ENERgy      Delta-E         GRMS    Step-size
MINI INTERN:          BONDs       ANGLes       UREY-b    DIHEdrals    IMPRopers
MINI CROSS:           CMAPs        PMF1D        PMF2D        PRIMO
MINI EXTERN:        VDWaals         ELEC       HBONds          ASP         USER
 ----------       ---------    ---------    ---------    ---------    ---------
MINI>        0   -933.41905      0.00000     11.33272      0.02000
MINI INTERN>      377.55480    796.66131    118.33669   2852.12949     58.68208
MINI CROSS>       -77.30932      0.00000      0.00000      0.00000
MINI EXTERN>     -468.70611  -4590.76800      0.00000      0.00000      0.00000
 ----------       ---------    ---------    ---------    ---------    ---------
MINI>       10  -2175.21989   1241.80084      7.24970  

## Sync Coord with pyCHARMM
We need to update the coords of crimm protein after minimization

In [19]:
## This is the new API for crimm sync coordinates with CHARMM
## The old sync_coord only works in a limited number of situations thus is DEPRECATED
fetch_coords_from_charmm(model.protein+model.ligand+model.phos_ligand+model.co_solvent)

## Solvation

In [20]:
solvator = Solvator(model)
# we want to keep the crystallograpic water using remove_existing_water=False
# doc string available for Solvator and
added_water = solvator.solvate(
    cutoff=8.0, solvcut=2.1, remove_existing_water=False, orient_coords=False
)
# Solvator.add_balancing_ions will add either sodium (SOD) or chloride (CLA)
# to the solvent according to the total charge of the system
balancing_ion_chain = solvator.add_balancing_ions()

Total charges before adding ions: -11.0
  [Chain A] -15.0
  [Chain B] 0.0
  [Chain C] 0.0
  [Chain D] 4.0


## Doc Strings for Solvator

In [21]:
Solvator?

Init signature: Solvator(entity) -> None
Docstring:     
Solvates a Structure, Model, or Chain level entity with water molecules.
The solvated entity will be returned as a Model level entity. The solvated
entity will be centered in a cubic box with side length equal to the
maximum dimension of the entity plus the cutoff distance. (i.e., Coordinates 
will be oriented using CoordManipulator.orient_coords() before solvation.)
The solvcut distance is the distance from the solute at which water
molecules will be removed. The solvcut distance is used to remove water 
molecules that are too close to the solute. 
If altloc atoms exist in the entity, the first altloc atoms will be used to
determine water molecules location during solvation.

Parameters
----------
entity : Structure, Model, or Chain level entity
    The entity to solvate. If a Structure level entity is provided, the
    first Model will be solvated. If a Model level entity is provided, all 
    chains in the model will be solvat

In [22]:
Solvator.solvate?

Signature:
Solvator.solvate(
    self,
    cutoff=9.0,
    solvcut=2.1,
    remove_existing_water=True,
    orient_coords=True,
) -> crimm.StructEntities.Model.Model
Docstring:
Solvates the entity and returns a Model level entity. The solvated
entity will be centered in a cubic box with side length equal to the
maximum dimension of the entity plus the cutoff distance. (i.e.,
Coordinates will be oriented using CoordManipulator.orient_coords()
before solvation.) The solvcut distance is the distance from the solute
at which water molecules will be removed. The solvcut distance is used
to remove water molecules that are too close to the solute. If altloc
atoms exist in the entity, the first altloc atoms will be used to
determine water molecules location during solvation.

Parameters
----------
cutoff : float, optional
    The distance from the solute to the edge of the cubic box. The
    default is 9.0.
solvcut : float, optional
    The distance from the solute at which water molecules wil

In [23]:
Solvator.add_balancing_ions?

Signature:
Solvator.add_balancing_ions(
    self,
    present_charge=None,
    cation='SOD',
    anion='CLA',
    skip_undefined=True,
) -> crimm.StructEntities.Chain.Ion
Docstring:
Add balancing ions to the solvated entity to bring total charge to zero.
The default cation is Na+ and the default anion is Cl-. If the entity is
not a solvated entity, a ValueError will be raised. A random selection of
water molecules in the water box will be replaced with balancing ions.
Returns a chain containing the balancing ions.

Parameters
----------
entity : Structure, Model, or Chain level entity
    The solvated entity to add balancing ions to.
present_charge : int, optional
    The present charge of the solvated entity. If None, the charge will be
    calculated from the entity. The default is None. If for any reason you
    want to balance the charge to a non-zero value, you can specify it here.
cation : str, optional
    The cation to use. The default is 'SOD' (Na+).
anion : str, optional
    

## Model after Solvation

Note that we have added 4 chloride ions as chain IA since the total charge was **+4**.<br>
The water box has to be splitted into two chains due to PDB residue number limit to **9999**.

In [24]:
model

NGLWidget()

<OrganizedModel model=5IGV Polypeptide(L)=1 Solvent=3 NucleosidePhosphate=1 Ligand=1 Ion=2 >
	│
	├───<Polypeptide(L) id=A Residues=298>
	├──────Description: Macrolide 2'-phosphotransferase II
	│
	├───<NucleosidePhosphate id=B Molecules=1>
	├──────Residue ID(s): GDP
	├──────Description: GUANOSINE-5'-DIPHOSPHATE
	│
	├───<Ligand id=C Molecules=1>
	├──────Residue ID(s): ZIT
	├──────Description: AZITHROMYCIN
	│
	├───<Ion id=D Residues=2>
	├──────Residue ID(s): CA, MG
	├──────Description: MAGNESIUM ION, CALCIUM ION
	│
	├───<Solvent id=E Residues=455>
	├──────Residue ID(s): HOH
	├──────Description: water
	│
	├───<Solvent id=WA Residues=9992>
	├──────Description: water
	│
	├───<Solvent id=WB Residues=2560>
	├──────Description: water
	│
	├───<Ion id=IA Residues=11>
	├──────Description: balancing ions (SOD)


## Load other Entities into CHARMM
Currently Ligand, Nucleophosphate, and CoSolvent can be automatically generated by TopologyLoader. <br>
Thus, if they exist, they can be safely loaded into CHARMM

In [25]:
load_ions(model.ion)
# This loads both crystallographic water and the water box generated by Solvator
load_water(model.solvent)
# Since crimm cannot build hydrogens on the crystallographic water yet
# we will build them in CHARMM and copy their coords to crimm
create_water_hs_from_charmm(model)

[crimm] Loading ion chain IO00
  
 CHARMM>     read sequence pdb -
 CHARMM>     name /tmp/tmpdf599gtu
 VOPEN> Attempting to open::/tmp/tmpdf599gtu::
 MAINIO> Sequence information being read from unit  91.
 TITLE>  *

          RESIDUE SEQUENCE --     2 RESIDUES
          MG      CAL     
 VCLOSE: Closing unit   91 with status "KEEP"
  
 CHARMM>     
  
 NO PATCHING WILL BE DONE ON THE FIRST RESIDUE
 NO PATCHING WILL BE DONE ON THE LAST  RESIDUE
 GENPSF> Segment   4 has been generated. Its identifier is IO00.
 PSFSUM> PSF modified: NONBOND lists and IMAGE atoms cleared.
 PSFSUM> Summary of the structure file counters :
         Number of segments      =        4   Number of residues   =      302
         Number of atoms         =     4952   Number of groups     =     1421
         Number of bonds         =     5012   Number of angles     =     9084
         Number of dihedrals     =    13299   Number of impropers  =      843
         Number of cross-terms   =      298   Number of autoge

In [26]:
# We can visualize the crystallographic water and see the hydrogens are added
model.solvent[0]

NGLWidget()

<Solvent id=E Residues=455>
  Residue ID(s): HOH
  Description: water


## Set up PBC and Minimize Water

In [27]:
# organize segids and ion types for image and cons_fix
non_solvent_segids = set()
all_ion_types = set()
for chain in model:
    if chain.chain_type == 'Solvent':
        continue
    elif chain.chain_type == 'Ion':
        for res in chain:
            all_ion_types.add(res.resname)
    else:
        for res in chain:
            non_solvent_segids.add(res.segid)

In [28]:
# anything but solvent or ions in the model
non_solvent_segids

{'LG00', 'LG01', 'PROA'}

In [29]:
# all types of ions loaded in pyCHARMM by crimm
all_ion_types

{'CAL', 'MG', 'SOD'}

In [30]:
# CHARMM scripting: crystal define cubic @boxsize @boxsize @boxsize 90 90 90
crystal.define_cubic(solvator.box_dim)
# CHARMM scripting: crystal build cutoff @boxhalf noper 0
crystal.build(solvator.box_dim/2)

 Crystal Parameters : Crystal Type = CUBI
           A     =   77.04339 B    =   77.04339 C     =   77.04339
           Alpha =   90.00000 Beta =   90.00000 Gamma =   90.00000
 XBUILD> Building all transformations with a minimum atom-atom
         contact distance of less than   38.52 Angstroms.

 Range of Grid Search for Transformation     1 :
 Lattice Vector A    -2 TO     2
 Lattice Vector B    -2 TO     2
 Lattice Vector C    -2 TO     2


 The number of transformations generated =    26


 Number  Symop   A   B   C   Distance

      1      1  -1  -1  -1     2.6826
      2      1  -1   0  -1     2.0953
      3      1  -1   1  -1     3.3216
      4      1   0  -1  -1     2.4960
      5      1   0   0  -1     1.6094
      6      1   0   1  -1     2.7724
      7      1  -1  -1   0     2.4622
      8      1  -1   0   0     1.8045
      9      1  -1   1   0     3.0165
     10      1   0  -1   0     1.6793
     11      1   0   1   0     1.6793
     12      1  -1  -1   1     3.5658
     1

1

In [31]:
# Turn on image centering - bysegment for protein, by residue for solvent and ions
# CHARMM scripting: image byseg xcen 0 ycen 0 zcen 0 select segid SEGID end
for segid in non_solvent_segids:
    image.setup_segment(0.0, 0.0, 0.0, segid)
# CHARMM scripting: image byres xcen 0 ycen 0 zcen 0 select resname tip3 end
image.setup_residue(0.0, 0.0, 0.0, 'TIP3')
# CHARMM scripting: image byres xcen 0 ycen 0 zcen 0 select resname ion_type end
for ion_type in all_ion_types:
    image.setup_residue(0.0, 0.0, 0.0, ion_type)

 select>     43 atoms have been selected out of   43984
 IMAGE CENTERING ON FOR SOME ATOMS
 select>   4783 atoms have been selected out of   43984
 IMAGE CENTERING ON FOR SOME ATOMS
 select>    124 atoms have been selected out of   43984
 IMAGE CENTERING ON FOR SOME ATOMS
 select>  39021 atoms have been selected out of   43984
 IMAGE CENTERING ON FOR SOME ATOMS
 select>     11 atoms have been selected out of   43984
 IMAGE CENTERING ON FOR SOME ATOMS
 select>      1 atoms have been selected out of   43984
 IMAGE CENTERING ON FOR SOME ATOMS
 select>      1 atoms have been selected out of   43984
 IMAGE CENTERING ON FOR SOME ATOMS


In [32]:
# Now specify nonbonded cutoffs for solvated box
cutnb = min(solvator.box_dim/2, 12)
cutim = cutnb
ctofnb = cutnb - 1.0
ctonnb = cutnb - 3.0

# Another nbonds example
# CHARMM scripting: nbonds cutnb @cutnb cutim @cutim ctofnb @ctofnb ctonnb @ctonnb -
#        inbfrq -1 imgfrq -1
non_bonded_script = pycharmm.NonBondedScript(
    cutnb=cutnb, cutim=cutim, ctonnb=ctonnb, ctofnb=ctofnb,
    eps=1.0,
    cdie=True,
    atom=True, vatom=True,
    fswitch=True, vfswitch=True,
    inbfrq=-1, imgfrq=-1
)

In [33]:
# We want to fix the protein and ligands and minimize the solvent to "fit"
# Select everything but solvent and ions
cons_fix_atoms = pycharmm.SelectAtoms()
for segid in non_solvent_segids:
    cons_fix_atoms |= pycharmm.SelectAtoms(seg_id=segid)

# Minimize the solvent positions with periodic boundary conditions using steepest descents
ener_dict = sd_minimize(200, non_bonded_script, cons_fix_selection=cons_fix_atoms)

  
 CHARMM>     nbonds cutnb 12 -
 CHARMM>     cutim 12 -
 CHARMM>     ctonnb 9.0 -
 CHARMM>     ctofnb 11.0 -
 CHARMM>     eps 1.0 -
 CHARMM>     cdie -
 CHARMM>     atom -
 CHARMM>     vatom -
 CHARMM>     fswitch -
 CHARMM>     vfswitch -
 CHARMM>     inbfrq -1 -
 CHARMM>     imgfrq -1

 SELECTED IMAGES ATOMS BEING CENTERED ABOUT  0.000000  0.000000  0.000000

 <MKIMAT2>: updating the image atom lists and remapping
 Transformation   Atoms  Groups  Residues  Min-Distance
    1  N1N1N1R1 has      63      21      21        1.52
    2  N1Z0N1R1 has     684     228     228        1.51
    3  N1P1N1R1 has      75      25      25        2.08
    4  Z0N1N1R1 has     765     255     255        1.23
    5  Z0Z0N1R1 has    6846    2284    2284        0.07
    6  Z0P1N1R1 has     750     250     250        1.22
    7  N1N1Z0R1 has     663     221     221        0.90
    8  N1Z0Z0R1 has    6640    2208    2191        0.05
    9  N1P1Z0R1 has     744     248     248        1.63
   10  Z0N1Z0R1 ha

/home/truman/crimm/crimm/Adaptors/pyCHARMMAdaptors.py:457: UserWarning: Atom fix constraint setup success: True
  warnings.warn(f"Atom fix constraint setup success: {has_cons_fix}")



 General atom nonbond list generation found:
 11880792 ATOM PAIRS WERE FOUND FOR ATOM LIST
   812450 GROUP PAIRS REQUIRED ATOM SEARCHES

 SPACE FOR  7277414 ATOM PAIRS AND        0 GROUP PAIRS

 Image nonbond list generation found:
  2256852 ATOM PAIRS WERE FOUND FOR ATOM LIST
        0 ATOM PAIRS WERE FOUND FOR ATOM SELF LIST
   224767 GROUP PAIRS REQUIRED ATOM SEARCHES

 PRNHBD: CUToff Hydrogen Bond  distance =    0.5000   Angle =   90.0000
         CuT switching ON HB dist. =     3.5000  OFf HB dist. =    4.0000
         CuT switching ON Hb Angle =    50.0000  OFf Hb Angle =   70.0000
         ACCEptor antecedents included
         All hydrogen bonds for each hydrogen will be found
         Hydrogen bonds between excluded atoms will be kept

 HBFIND-exclusions:******* due to distance cutoff,       0 due to angle cutoff
                         0 primary donor to image acceptor hbonds found
 HBFIND-exclusions:******* due to distance cutoff,       0 due to angle cutoff
              

In [34]:
fetch_coords_from_charmm(model)
model

NGLWidget()

<OrganizedModel model=5IGV Polypeptide(L)=1 Solvent=3 NucleosidePhosphate=1 Ligand=1 Ion=2 >
	│
	├───<Polypeptide(L) id=A Residues=298>
	├──────Description: Macrolide 2'-phosphotransferase II
	│
	├───<NucleosidePhosphate id=B Molecules=1>
	├──────Residue ID(s): GDP
	├──────Description: GUANOSINE-5'-DIPHOSPHATE
	│
	├───<Ligand id=C Molecules=1>
	├──────Residue ID(s): ZIT
	├──────Description: AZITHROMYCIN
	│
	├───<Ion id=D Residues=2>
	├──────Residue ID(s): CA, MG
	├──────Description: MAGNESIUM ION, CALCIUM ION
	│
	├───<Solvent id=E Residues=455>
	├──────Residue ID(s): HOH
	├──────Description: water
	│
	├───<Solvent id=WA Residues=9992>
	├──────Description: water
	│
	├───<Solvent id=WB Residues=2560>
	├──────Description: water
	│
	├───<Ion id=IA Residues=11>
	├──────Description: balancing ions (SOD)


In [35]:
pcm_write.coor_card(f'{PDBID}.crd')
pcm_write.psf_card(f'{PDBID}.psf')

  
 CHARMM>     write name 5igv.crd -
 CHARMM>     coor card
 VOPEN> Attempting to open::5igv.crd::
 RDTITL>  
 RDTITL> No title read.
 VCLOSE: Closing unit   91 with status "KEEP"
 VCLOSE: Closing unit   91 with status "KEEP"
  
 CHARMM>     
  
  
 CHARMM>     write name 5igv.psf -
 CHARMM>     psf card
 VOPEN> Attempting to open::5igv.psf::
 RDTITL>  
 RDTITL> No title read.
 VCLOSE: Closing unit   91 with status "KEEP"
 VCLOSE: Closing unit   91 with status "KEEP"
  
 CHARMM>     
  
